In [3]:
from conllu import parse
from conllu.models import TokenList, Token

In [32]:
def get_children_tokens(token_head: Token, group_token: TokenList):
    children_tokens = []
    for token in group_token:
        if token['head'] == token_head['id'] and token['deprel'] != '_':
            children_tokens.append(token)
            children_tokens.extend(get_children_tokens(token, group_token))

    return children_tokens

In [33]:
def get_list_one_step_children_token(token_head: Token, group_token: TokenList):
    one_step_children_tokens = []
    for token in group_token:
        if token['head'] == token_head['id'] and token['deprel'] != '_':
            one_step_children_tokens.append(token)

    return one_step_children_tokens

In [34]:
def get_phrase_tokens(token_head: Token, group_token: TokenList):
    phrase_tokens = []
    for token in group_token:
        if token['head'] == token_head['id'] and token['deprel'] == '_':
            if token['form'] != '_':
                phrase_tokens.append(token)
            else:
                phrase_tokens.extend(get_phrase_tokens(token, group_token))
            phrase_tokens.extend(get_children_tokens(token, group_token))

    return phrase_tokens

In [35]:
def get_pairs(group_token: TokenList) -> list:
    pairs = []
    for token in group_token:
        if token['head'] != 0 and token['head'] != 1:
            if token['id'] < token['head']:
                pairs.append([token['id'], token['head']])
            else:
                pairs.append([token['head'], token['id']])
    return pairs

In [36]:
def create_ssg(id: int, upos: str, xpos: str, head: int, deprel: str) -> Token:
    ssg_token = Token(
        id = id,
        form = '_',
        lemma = '_',
        upos = upos,
        xpos = xpos,
        feats = None,
        head = head,
        deprel = deprel,
        deps = None,
        misc = None
        )

    return ssg_token

In [9]:
def _add_root_token(group_token: TokenList) -> TokenList:
    sentence = ""
    for token in group_token:
        sentence += token['form'] + ' '
        if token['head'] == 0:
            token['head'] = len(group_token) + 1
    sentence = sentence[:-1]

    root_token = create_ssg(len(group_token) + 1, sentence, '*', 0, '_')
    group_token.append(root_token)

    return group_token

In [37]:
def add_root_token(group_token: TokenList) -> TokenList:
    sentence = ""
    for token in group_token:
        sentence += token['form'] + ' '
        token['id'] += 1
        token['head'] += 1
    sentence = sentence[:-1]

    root_token = create_ssg(1, sentence, '*', 0, '_')
    group_token.append(root_token)

    group_token.sort(key=lambda token: token['id'])

    return group_token

In [38]:
def check_feat_token(token: Token, feat: str) -> bool:
    if token['feats'] != None:
        if feat in str(list(token['feats'].keys())[0]):
            return True
    return False

In [12]:
data = open("input_test.conllu", mode="r", encoding="utf-8")

sentences = parse(data.read())

group_token = sentences[0]

group_token = add_root_token(group_token)

sentences[0] = group_token

with open('output_test.conllu', mode="w", encoding="utf-8") as f:
    f.writelines([sentence.serialize() + "\n" for sentence in sentences])

In [13]:
data = open("input_test.conllu", mode="r", encoding="utf-8")

sentences = parse(data.read())

#for i in range(len(sentences)):
sentence = sentences[0]
sentence = add_root_token(sentence)

In [14]:
bool_new_group = False
for token in sentence:
    if token['upos'] == 'VERB' and not check_feat_token(token, 'ИНФ'):
        verb_token = token
        list_one_step_children_token = get_list_one_step_children_token(token, sentence)
        for child_token in list_one_step_children_token:
            if child_token['upos'] == 'NOUN':
                group = [verb_token, child_token]

                pairs = get_pairs(sentence)
                noun_pairs = []
                for pair in pairs:
                    if child_token['id'] in pair and child_token['head'] not in pair:
                        noun_pairs.append(pair)
                        pairs.remove(pair)
                        print(pair)

                qnt_noun_pairs = len(noun_pairs)
                for noun_pair in noun_pairs:
                    if child_token['head'] > noun_pair[0] and child_token['head'] < noun_pair[1]:
                        print('noun_pair:', noun_pair)
                        print('====================')
                        noun_pairs.remove(noun_pair)
                        continue
                    for pair in pairs:
                        if (noun_pair[0] > pair[0] and noun_pair[0] < pair[1] and (noun_pair[1] < pair[0] or noun_pair[1] > pair[1])) or (noun_pair[1] > pair[0] and noun_pair[1] < pair[1] and (noun_pair[0] < pair[0] or noun_pair[0] > pair[1])):
                            print('pair:', pair)
                            print('inf_pair:', inf_pair)
                            noun_pairs.remove(noun_pair)

                if qnt_noun_pairs > len(noun_pairs):
                    bool_new_group = True

                    group_token = []
                    group_token.append(sentence[child_token['head'] - 1])
                    group_token.append(child_token)
                    for noun_pair in noun_pairs:
                        if noun_pair[0] != child_token['id']:
                            group_token.append(sentence[noun_pair[0] - 1])
                        else:
                            group_token.append(sentence[noun_pair[1] - 1])
                    #print(group_token)

                    #получить текст группы
                    group_token.sort(key=lambda token: token['id'])
                    phrase = ''
                    for token in group_token:
                        phrase += token['form'] + ' '
                    phrase = phrase[:-1]

                    list_one_step_children_token = []
                    for token in group_token:
                        list_one_step_children_token.extend(get_list_one_step_children_token(token, sentence))

                    for token in list_one_step_children_token:
                        if token not in group_token:
                            #print(token)
                            token['head'] = len(sentence) + 1
                    sentence.append(create_ssg(len(sentence) + 1, phrase, 'Г_6', verb_token['head'], '_'))
                    verb_token['head'] = len(sentence)

    ##if not bool_new_group:
    #    pass

    if check_feat_token(token, 'ИНФ') and not check_feat_token(sentence[token['head'] - 1], 'ИНФ'):
        inf_token = token
        #print(inf_token)

        group_token = []
        head_token = inf_token
        inf_group_token = []
        print('222')
        while head_token['upos'] in ['VERB', 'NOUN', 'ADJ', 'ADV']:
            print(head_token)
            print('111')
            root_token = head_token
            if check_feat_token(head_token, 'ИНФ'):
                #print(head_token)
                inf_group_token.append(head_token)
            #print(head_token)
            group_token.append(head_token)
            head_token = sentence[head_token['head'] - 1]
            print(head_token['head'])
            #print(head_token)

        child_token = inf_token
        bool_child = True
        while bool_child:
            bool_child = False
            for token in sentence:
                if token['head'] == child_token['id']:
                    if check_feat_token(token, 'ИНФ'):
                        #print(token)
                        inf_group_token.append(token)
                        group_token.append(token)
                        child_token = token
                        bool_child = True
                        break
        list_one_step_children_token = get_list_one_step_children_token(child_token, sentence)
        bool_noun = False
        for child_token in list_one_step_children_token:
            if child_token['upos'] == 'NOUN':
                noun_token = child_token
                bool_noun = True
                group_token.append(noun_token)
                break

        if bool_noun:
            pairs = get_pairs(sentence)
            noun_pairs = []
            for pair in pairs:
                if child_token['id'] in pair and child_token['head'] not in pair:
                    noun_pairs.append(pair)
                    pairs.remove(pair)
                    #print(pair)

            qnt_noun_pairs = len(noun_pairs)
            for noun_pair in noun_pairs:
                if child_token['head'] > noun_pair[0] and child_token['head'] < noun_pair[1]:
                    #print('noun_pair:', noun_pair)
                    #print('====================')
                    noun_pairs.remove(noun_pair)
                    continue
                for pair in pairs:
                    if (noun_pair[0] > pair[0] and noun_pair[0] < pair[1] and (noun_pair[1] < pair[0] or noun_pair[1] > pair[1])) or (noun_pair[1] > pair[0] and noun_pair[1] < pair[1] and (noun_pair[0] < pair[0] or noun_pair[0] > pair[1])):
                        #print('pair:', pair)
                        #print('inf_pair:', inf_pair)
                        noun_pairs.remove(noun_pair)

            if qnt_noun_pairs > len(noun_pairs):
                bool_new_group = True

                group_token.sort(key=lambda token: token['id'])
                phrase = ''
                for token in group_token:
                    phrase += token['form'] + ' '
                phrase = phrase[:-1]
                #print(phrase)

                list_one_step_children_token = []
                for token in group_token:
                    list_one_step_children_token.extend(get_list_one_step_children_token(token, sentence))

                #list_one_step_children_token = []
                #for token in group_token:
                #    for buf_token in get_list_one_step_children_token(token, sentence):
                #        if buf_token not in group_token:
                #            list_one_step_children_token.extend(buf_token)

                for token in list_one_step_children_token:
                    if token not in group_token:
                        #print(token)
                        token['head'] = len(sentence) + 1
                sentence.append(create_ssg(len(sentence) + 1, phrase, 'Г_6', root_token['head'], '_'))
                root_token['head'] = len(sentence)

NameError: name 'get_list_one_step_children_token' is not defined

In [15]:
with open('output_test.conllu', mode="w", encoding="utf-8") as f:
    f.writelines([sentence.serialize() + "\n" for sentence in sentences])

In [16]:
def rule_g_5(sentence: TokenList):
    for token in sentence:
        bool_new_group = False

        if check_feat_token(token, 'ИНФ') and not check_feat_token(sentence[token['head'] - 1], 'ИНФ'):
            inf_token = token
            #print(inf_token)

            group_token = []
            head_token = inf_token
            inf_group_token = []
            while head_token['upos'] in ['VERB', 'NOUN', 'ADJ', 'ADV']:
                root_token = head_token
                if check_feat_token(head_token, 'ИНФ'):
                    #print(head_token)
                    inf_group_token.append(head_token)
                #print(head_token)
                group_token.append(head_token)
                head_token = sentence[head_token['head'] - 1]

            child_token = inf_token
            bool_child = True
            while bool_child:
                bool_child = False
                for token in sentence:
                    if token['head'] == child_token['id']:
                        if check_feat_token(token, 'ИНФ'):
                            print(token)
                            inf_group_token.append(token)
                            group_token.append(token)
                            child_token = token
                            bool_child = True
                            break

            pairs = get_pairs(sentence)
            #print(pairs)
            inf_pairs = []
            for token in inf_group_token:
                for pair in pairs:
                    #print(pair)
                    if token['id'] in pair and token['head'] not in pair:
                        inf_pairs.append(pair)
                        pairs.remove(pair)
            #print(inf_pairs)
            #print(pairs)

            print('pairs:', pairs)
            print('inf_pairs:', inf_pairs)
            qnt_inf_pairs = len(inf_pairs)
            for inf_pair in inf_pairs:
                if inf_token['head'] > inf_pair[0] and inf_token['head'] < inf_pair[1]:
                    print('inf_pair:', inf_pair)
                    print('====================')
                    inf_pairs.remove(inf_pair)
                    continue
                for pair in pairs:
                    if (inf_pair[0] > pair[0] and inf_pair[0] < pair[1] and (inf_pair[1] < pair[0] or inf_pair[1] > pair[1])) or (inf_pair[1] > pair[0] and inf_pair[1] < pair[1] and (inf_pair[0] < pair[0] or inf_pair[0] > pair[1])):
                        print('pair:', pair)
                        print('inf_pair:', inf_pair)
                        inf_pairs.remove(inf_pair)

            print('inf_pairs:', inf_pairs)
            if qnt_inf_pairs > len(inf_pairs):
                bool_new_group = True

                print(len(group_token))
                for inf_pair in inf_pairs:
                    print(inf_pair)
                    bool_inf_0 = True
                    bool_inf_1 = True
                    for inf_token in inf_group_token:
                        if inf_pair[0] == inf_token['id']:
                            bool_inf_0 = False
                        if inf_pair[1] == inf_token['id']:
                            bool_inf_1 = False
                    if bool_inf_0:
                        group_token.append(sentence[inf_pair[0] - 1])
                    if bool_inf_1:
                        group_token.append(sentence[inf_pair[1] - 1])
                print(len(group_token))

                group_token.sort(key=lambda token: token['id'])
                phrase = ''
                for token in group_token:
                    phrase += token['form'] + ' '
                phrase = phrase[:-1]
                print(phrase)

                list_one_step_children_token = []
                for token in group_token:
                    list_one_step_children_token.extend(get_list_one_step_children_token(token, sentence))

                #list_one_step_children_token = []
                #for token in group_token:
                #    for buf_token in get_list_one_step_children_token(token, sentence):
                #        if buf_token not in group_token:
                #            list_one_step_children_token.extend(buf_token)

                for token in list_one_step_children_token:
                    if token not in group_token:
                        print(token)
                        token['head'] = len(sentence) + 1
                sentence.append(create_ssg(len(sentence) + 1, phrase, 'Г_5', root_token['head'], '_'))
                root_token['head'] = len(sentence)

    return bool_new_group


In [17]:
def rule_g_3(sentence: TokenList):
    bool_new_group = False
    for token in sentence:
        #Поиск ГЛ, который не является ИНФ
        if token['upos'] == 'VERB' and not check_feat_token(token, 'ИНФ'):
            verb_token = token

            #Поиск ИНФ, который зависит от найденного ГЛ
            list_one_step_children_token = get_list_one_step_children_token(verb_token, sentence)
            inf_token = None
            for child_token in list_one_step_children_token:
                if check_feat_token(child_token, 'ИНФ'):
                    inf_token = child_token
                    break
            if inf_token == None:
                continue

            #Поиск двух наборов пар связей (в первой все связи, в которых отсуствует найденный ИНФ, а во второй - присуствует)
            pairs = get_pairs(sentence)
            inf_pairs = []
            for pair in pairs:
                if inf_token['id'] in pair and inf_token['head'] not in pair:
                    inf_pairs.append(pair)
                    pairs.remove(pair)

            #Удалить все пары с ИНФ, которые нарушают проективность
            qnt_inf_pairs = len(inf_pairs)
            for inf_pair in inf_pairs:
                if inf_token['head'] > inf_pair[0] and inf_token['head'] < inf_pair[1]:
                    inf_pairs.remove(inf_pair)
                    continue
                for pair in pairs:
                    if (inf_pair[0] > pair[0] and inf_pair[0] < pair[1] and (inf_pair[1] < pair[0] or inf_pair[1] > pair[1])) or (inf_pair[1] > pair[0] and inf_pair[1] < pair[1] and (inf_pair[0] < pair[0] or inf_pair[0] > pair[1])):
                        inf_pairs.remove(inf_pair)

            #Проверка, что нужно добавлять новую группу
            if qnt_inf_pairs > len(inf_pairs):
                bool_new_group = True

                #Получить набор слов, которые образуют новую группу
                group_token = [verb_token, inf_token]
                for inf_pair in inf_pairs:
                    if inf_pair[0] != inf_token['id']:
                        group_token.append(sentence[inf_pair[0] - 1])
                    else:
                        group_token.append(sentence[inf_pair[1] - 1])

                #Получить текст группы
                group_token.sort(key=lambda token: token['id'])
                phrase = ''
                for token in group_token:
                    phrase += token['form'] + ' '
                phrase = phrase[:-1]

                #Получить все слова, которые связаны со словами из группы
                list_one_step_children_token = []
                for token in group_token:
                    list_one_step_children_token.extend(get_list_one_step_children_token(token, sentence))

                #Добавить новую группу и изменить связи между элементами ССГ
                for token in list_one_step_children_token:
                    if token not in group_token:
                        token['head'] = len(sentence) + 1
                sentence.append(create_ssg(len(sentence) + 1, phrase, 'Г3', verb_token['head'], '_'))
                verb_token['head'] = len(sentence)

    return bool_new_group

In [16]:
def rule_g_4(sentence: TokenList):
    bool_new_group = False
    for token in sentence:
        #поиск ГЛ, который не является ИНФ
        if token['upos'] in ['NOUN', 'ADJ', 'ADV']:
            id_head_token = token['id']
            #print('1', token)
            root_token = token

            #if sentence[token['head'] - 1]['form'] == '_':
            #    root_token = sentence[token['head'] - 1]
            #    print('2', root_token['upos'])

            #поиск ИНФ, зависимого от найденного слова
            list_one_step_children_token = get_list_one_step_children_token(root_token, sentence)
            inf_token = None
            for child_token in list_one_step_children_token:
                if check_feat_token(child_token, 'ИНФ'):
                    inf_token = child_token
                    #print(inf_token)
                    break
            if inf_token == None:
                continue

            #поиск двух наборов пар связей (в первой все связи, в которых отсуствует найденный ИНФ, а во второй - присуствует)
            pairs = get_pairs(sentence)
            inf_pairs = []
            for pair in pairs:
                if inf_token['id'] in pair and inf_token['head'] not in pair:
                    inf_pairs.append(pair)
                    pairs.remove(pair)
            #print(pairs)
            #print(inf_pairs)
            #print(inf_token)

            #удалить все пары с ИНФ, которые нарушают проективность
            qnt_inf_pairs = len(inf_pairs)
            for inf_pair in inf_pairs:
                if id_head_token > inf_pair[0] and id_head_token < inf_pair[1]:
                    #print('inf_pair:', inf_pair)
                    #print('====================')
                    inf_pairs.remove(inf_pair)
                    continue
                for pair in pairs:
                    if (inf_pair[0] > pair[0] and inf_pair[0] < pair[1] and (inf_pair[1] < pair[0] or inf_pair[1] > pair[1])) or (inf_pair[1] > pair[0] and inf_pair[1] < pair[1] and (inf_pair[0] < pair[0] or inf_pair[0] > pair[1])):
                        #print('pair:', pair)
                        #print('inf_pair:', inf_pair)
                        inf_pairs.remove(inf_pair)

            #проверка, что нужно добавлять новую группу
            if qnt_inf_pairs > len(inf_pairs):
                bool_new_group = True

                #получить группу слов, которые образуют новую группу
                group_token = []
                group_token.append(sentence[inf_token['head'] - 1])
                group_token.append(inf_token)
                for inf_pair in inf_pairs:
                    if inf_pair[0] != inf_token['id']:
                        group_token.append(sentence[inf_pair[0] - 1])
                    else:
                        group_token.append(sentence[inf_pair[1] - 1])
                #print('rrr')
                #print(group_token)

                #получить текст группы
                #for token in group_token:
                #    print('t', token)
                #    if token['form'] == '_':
                #        group_token.append(get_list_token_from_phrase(token, sentence))
                group_token.sort(key=lambda token: token['id'])
                phrase = ''
                for token in group_token:
                    phrase += token['form'] + ' '
                phrase = phrase[:-1]

                #получить все слова, которые связаны со словами из группы
                list_one_step_children_token = []
                for token in group_token:
                    list_one_step_children_token.extend(get_list_one_step_children_token(token, sentence))

                #добавить новую группу и изменить связи между элементами ССГ
                for token in list_one_step_children_token:
                    if token not in group_token:
                        #print(token)
                        token['head'] = len(sentence) + 1
                sentence.append(create_ssg(len(sentence) + 1, phrase, 'Г_4', root_token['head'], '_'))
                root_token['head'] = len(sentence)

    return bool_new_group

In [17]:
def rule_g_5(sentence: TokenList):
    bool_new_group = False
    for token in sentence:

        if check_feat_token(token, 'ИНФ') and not check_feat_token(sentence[token['head'] - 1], 'ИНФ'):
            inf_token = token
            #print(inf_token)

            group_token = []
            head_token = inf_token
            inf_group_token = []
            while head_token['upos'] in ['VERB', 'NOUN', 'ADJ', 'ADV']:
                root_token = head_token
                if check_feat_token(head_token, 'ИНФ'):
                    #print(head_token)
                    inf_group_token.append(head_token)
                #print(head_token)
                group_token.append(head_token)
                head_token = sentence[head_token['head'] - 1]
                #print(head_token)


            child_token = inf_token
            bool_child = True
            while bool_child:
                bool_child = False
                for token in sentence:
                    if token['head'] == child_token['id']:
                        if check_feat_token(token, 'ИНФ'):
                            #print(token)
                            inf_group_token.append(token)
                            group_token.append(token)
                            child_token = token
                            bool_child = True
                            break

            if len(group_token) >= 3:
                pairs = get_pairs(sentence)
                #print(pairs)
                inf_pairs = []
                for token in inf_group_token:
                    for pair in pairs:
                        #print(pair)
                        if token['id'] in pair and token['head'] not in pair:
                            inf_pairs.append(pair)
                            pairs.remove(pair)
                #print(inf_pairs)
                #print(pairs)

                #print('pairs:', pairs)
                #print('inf_pairs:', inf_pairs)
                qnt_inf_pairs = len(inf_pairs)
                for inf_pair in inf_pairs:
                    if inf_token['head'] > inf_pair[0] and inf_token['head'] < inf_pair[1]:
                        #print('inf_pair:', inf_pair)
                        #print('====================')
                        inf_pairs.remove(inf_pair)
                        continue
                    for pair in pairs:
                        if (inf_pair[0] > pair[0] and inf_pair[0] < pair[1] and (inf_pair[1] < pair[0] or inf_pair[1] > pair[1])) or (inf_pair[1] > pair[0] and inf_pair[1] < pair[1] and (inf_pair[0] < pair[0] or inf_pair[0] > pair[1])):
                            #print('pair:', pair)
                            #print('inf_pair:', inf_pair)
                            inf_pairs.remove(inf_pair)

                #print('inf_pairs:', inf_pairs)
                if qnt_inf_pairs > len(inf_pairs):
                    bool_new_group = True

                    #print(len(group_token))
                    for inf_pair in inf_pairs:
                        #print(inf_pair)
                        bool_inf_0 = True
                        bool_inf_1 = True
                        for inf_token in inf_group_token:
                            if inf_pair[0] == inf_token['id']:
                                bool_inf_0 = False
                            if inf_pair[1] == inf_token['id']:
                                bool_inf_1 = False
                        if bool_inf_0:
                            group_token.append(sentence[inf_pair[0] - 1])
                        if bool_inf_1:
                            group_token.append(sentence[inf_pair[1] - 1])
                    #print(len(group_token))

                    group_token.sort(key=lambda token: token['id'])
                    phrase = ''
                    for token in group_token:
                        phrase += token['form'] + ' '
                    phrase = phrase[:-1]
                    #print(phrase)

                    list_one_step_children_token = []
                    for token in group_token:
                        list_one_step_children_token.extend(get_list_one_step_children_token(token, sentence))

                    #list_one_step_children_token = []
                    #for token in group_token:
                    #    for buf_token in get_list_one_step_children_token(token, sentence):
                    #        if buf_token not in group_token:
                    #            list_one_step_children_token.extend(buf_token)

                    for token in list_one_step_children_token:
                        if token not in group_token:
                            #print(token)
                            token['head'] = len(sentence) + 1
                    sentence.append(create_ssg(len(sentence) + 1, phrase, 'Г_5', root_token['head'], '_'))
                    root_token['head'] = len(sentence)

    return bool_new_group

In [30]:
def rule_g_6(sentence: TokenList):
    bool_new_group = False
    for token in sentence:
        if token['upos'] == 'VERB' and not check_feat_token(token, 'ИНФ'):
            verb_token = token
            list_one_step_children_token = get_list_one_step_children_token(token, sentence)
            for child_token in list_one_step_children_token:
                if child_token['upos'] == 'NOUN':
                    group = [verb_token, child_token]

                    pairs = get_pairs(sentence)
                    noun_pairs = []
                    for pair in pairs:
                        if child_token['id'] in pair and child_token['head'] not in pair:
                            noun_pairs.append(pair)
                            pairs.remove(pair)

                    qnt_noun_pairs = len(noun_pairs)
                    for noun_pair in noun_pairs:
                        if child_token['head'] > noun_pair[0] and child_token['head'] < noun_pair[1]:
                            noun_pairs.remove(noun_pair)
                            continue
                        for pair in pairs:
                            if (noun_pair[0] > pair[0] and noun_pair[0] < pair[1] and (noun_pair[1] < pair[0] or noun_pair[1] > pair[1])) or (noun_pair[1] > pair[0] and noun_pair[1] < pair[1] and (noun_pair[0] < pair[0] or noun_pair[0] > pair[1])):
                                noun_pairs.remove(noun_pair)

                    if qnt_noun_pairs > len(noun_pairs):
                        bool_new_group = True

                        group_token = []
                        group_token.append(sentence[child_token['head'] - 1])
                        group_token.append(child_token)
                        for noun_pair in noun_pairs:
                            if noun_pair[0] != child_token['id']:
                                group_token.append(sentence[noun_pair[0] - 1])
                            else:
                                group_token.append(sentence[noun_pair[1] - 1])

                        group_token.sort(key=lambda token: token['id'])
                        phrase = ''
                        for token in group_token:
                            phrase += token['form'] + ' '
                        phrase = phrase[:-1]

                        list_one_step_children_token = []
                        for token in group_token:
                            list_one_step_children_token.extend(get_list_one_step_children_token(token, sentence))

                        for token in list_one_step_children_token:
                            if token not in group_token:
                                #print(token)
                                token['head'] = len(sentence) + 1
                        sentence.append(create_ssg(len(sentence) + 1, phrase, 'Г_6', verb_token['head'], '_'))
                        verb_token['head'] = len(sentence)


        if check_feat_token(token, 'ИНФ') and not check_feat_token(sentence[token['head'] - 1], 'ИНФ'):
            inf_token = token

            group_token = []
            head_token = inf_token
            inf_group_token = []
            print('222')
            while head_token['upos'] in ['VERB', 'NOUN', 'ADJ', 'ADV']:
                print(head_token)
                print('111')
                root_token = head_token
                if check_feat_token(head_token, 'ИНФ'):
                    inf_group_token.append(head_token)
                group_token.append(head_token)
                head_token = sentence[head_token['head'] - 1]
                print(head_token['head'])

            child_token = inf_token
            bool_child = True
            while bool_child:
                bool_child = False
                for token in sentence:
                    if token['head'] == child_token['id']:
                        if check_feat_token(token, 'ИНФ'):
                            #print(token)
                            inf_group_token.append(token)
                            group_token.append(token)
                            child_token = token
                            bool_child = True
                            break
            list_one_step_children_token = get_list_one_step_children_token(child_token, sentence)
            bool_noun = False
            for child_token in list_one_step_children_token:
                if child_token['upos'] == 'NOUN':
                    noun_token = child_token
                    bool_noun = True
                    group_token.append(noun_token)
                    break

            if bool_noun:
                pairs = get_pairs(sentence)
                noun_pairs = []
                for pair in pairs:
                    if child_token['id'] in pair and child_token['head'] not in pair:
                        noun_pairs.append(pair)
                        pairs.remove(pair)

                qnt_noun_pairs = len(noun_pairs)
                for noun_pair in noun_pairs:
                    if child_token['head'] > noun_pair[0] and child_token['head'] < noun_pair[1]:
                        noun_pairs.remove(noun_pair)
                        continue
                    for pair in pairs:
                        if (noun_pair[0] > pair[0] and noun_pair[0] < pair[1] and (noun_pair[1] < pair[0] or noun_pair[1] > pair[1])) or (noun_pair[1] > pair[0] and noun_pair[1] < pair[1] and (noun_pair[0] < pair[0] or noun_pair[0] > pair[1])):
                            noun_pairs.remove(noun_pair)

                if qnt_noun_pairs > len(noun_pairs):
                    bool_new_group = True

                    group_token.sort(key=lambda token: token['id'])
                    phrase = ''
                    for token in group_token:
                        phrase += token['form'] + ' '
                    phrase = phrase[:-1]

                    list_one_step_children_token = []
                    for token in group_token:
                        list_one_step_children_token.extend(get_list_one_step_children_token(token, sentence))

                    for token in list_one_step_children_token:
                        if token not in group_token:
                            token['head'] = len(sentence) + 1
                    sentence.append(create_ssg(len(sentence) + 1, phrase, 'Г_6', root_token['head'], '_'))
                    root_token['head'] = len(sentence)

    return bool_new_group

In [19]:
def rule_g_15_16(sentence: TokenList):
    pronoun = ['Я','МЫ','ТЫ','ВЫ','ОН','ОНО','ОНА','ОНИ',
            'МЕНЯ','НАС','ТЕБЯ','ВАС','ЕЕ','ЕЙ','ЕЁ','ИХ',
            'МНЕ','ТЕБЕ','ВАМ','ЕМУ','ЕЁ','ИМ',
            'МЕНЯ','НАС','ВАС','ЕГО',
            'НАМИ','ТОБОЙ','ВАМИ','ЕЮ','ИМИ', 'СЕЙ']
    introductory_groups = []
    for token in sentence:
        bool_new_group = False
        group = []
        if token['deprel'] == 'вводн' and token['form'] != '_':
            if sentence[token['head'] - 1]['xpos'] not in ['Г15', 'Г16']:
                group = [token]
                group.extend(get_list_children_token(token, sentence))
                introductory_groups.append(group)

    introductory_groups_15 = []
    introductory_groups_16 = []
    for group in introductory_groups:
        bool_15 = True
        bool_16 = True
        for token in group:
            if token['deprel'] in ['предик']:
                bool_15 = False
                bool_16 = False
            if token['deprel'] in ['аппоз', 'об-аппоз']:
                bool_15 = False
            if not check_feat_token(token, 'ИМ') or check_feat_token(token, 'ЗВ'):
                bool_16 = False
            if token['upos'] in ['VERB']:
                bool_16 = False
            if token['lemma'] in pronoun:
                bool_15 = False
            if token['upos'] in ['ADJ'] and len(group) == 1:
                bool_15 = False
        if bool_15:
            bool_new_group = True
            introductory_groups_15.append(group)
        elif bool_16:
            bool_new_group = True
            introductory_groups_16.append(group)

    if bool_new_group:
        id_new_group = len(sentence) + 1

        check_group_15 = []
        for group in introductory_groups_15:
            for token in group:
                check_group_15.append(token)
        check_group_16 = []
        for group in introductory_groups_16:
            for token in group:
                check_group_16.append(token)

        group = []
        if introductory_groups_15 != []:
            for token in sentence:
                if token['head'] == 1:
                    token['head'] = id_new_group
                if token not in check_group_15 and token['form'] != '_':
                    group.append(token)

            phrase = ''
            group.sort(key=lambda token: token['id'])
            for token in group:
                phrase += token['form'] + ' '
            phrase = phrase[:-1]
            sentence.append(create_ssg(id_new_group, phrase, 'Г15', 1, '_'))
            id_15_group = len(sentence)

            for introductory_group in introductory_groups_15:
                if len(introductory_group) > 1:
                    introductory_group.sort(key=lambda token: token['id'])
                    introductory_group[0]['deprel'] = '_'
                    introductory_group[0]['head'] = len(sentence) + 1
                    phrase = ""
                    for token in introductory_group:
                        phrase += token['form'] + ' '
                    phrase = phrase[:-1]
                    sentence.append(create_ssg(len(sentence) + 1, phrase, 'Г15', id_new_group, 'вводн'))
                introductory_group[0]['head'] = len(sentence)

        group = []
        if introductory_groups_16 != []:
            if introductory_groups_15 != []:
                for token in sentence:
                    if token['head'] == id_new_group and token['deprel'] == '_':
                        token['head'] = len(sentence) + 1
                    if token not in check_group_16 and token not in check_group_15 and token['form'] != '_':
                        group.append(token)
                id_new_group = len(sentence) + 1
            else:
                for token in sentence:
                    if token['head'] == 1:
                            token['head'] = id_new_group
                    if token not in check_group_16 and token not in check_group_15 and token['form'] != '_':
                        group.append(token)

            phrase = ''
            group.sort(key=lambda token: token['id'])
            for token in group:
                phrase += token['form'] + ' '
            phrase = phrase[:-1]
            if introductory_groups_15 != []:
                sentence.append(create_ssg(id_new_group, phrase, 'Г16', id_15_group, '_'))
            else:
                sentence.append(create_ssg(id_new_group, phrase, 'Г16', 1, '_'))

            for introductory_group in introductory_groups_16:
                if len(introductory_group) > 1:
                    introductory_group.sort(key=lambda token: token['id'])
                    introductory_group[0]['deprel'] = '_'
                    introductory_group[0]['head'] = len(sentence) + 1
                    phrase = ""
                    for token in introductory_group:
                        phrase += token['form'] + ' '
                    phrase = phrase[:-1]
                    sentence.append(create_ssg(len(sentence) + 1, phrase, 'Г16', id_new_group, 'вводн'))
                introductory_group[0]['head'] = len(sentence)

    return bool_new_group

In [39]:
data = open("input_test.conllu", mode="r", encoding="utf-8")

sentences = parse(data.read())

#for i in range(len(sentences)):
group_token = sentences[0]

group_token = add_root_token(group_token)

bool_while = True
while bool_while:
    bool_while = False

    #bool_new_group = rule_g_15_16(group_token)
    if bool_new_group:
        bool_while = True

    #bool_new_group = rule_g_5(group_token)
    if bool_new_group:
        bool_while = True

    bool_new_group = rule_g_6(group_token)
    if bool_new_group:
        bool_while = True

    bool_new_group = rule_g_3(group_token)
    if bool_new_group:
        bool_while = True

    #bool_new_group = rule_g_4(group_token)
    if bool_new_group:
        bool_while = True

    print(bool_while)

with open('output_test.conllu', mode="w", encoding="utf-8") as f:
    f.writelines([sentence.serialize() + "\n" for sentence in sentences])

222
посещать.
111
1
хочет
111
0
True
222
посещать.
111
10
хочет
111
9
True
222
посещать.
111
10
хочет
111
9
False


In [18]:
data = open("input_test.conllu", mode="r", encoding="utf-8")

sentences = parse(data.read())

group_token = sentences[0]
group_token[0]
get_phrase_tokens(group_token[0], group_token)

[]

In [26]:
a = True
b = False
if a * b:
    print('yes')

In [26]:
def get_id_head_token(token: Token, sentence: TokenList) -> Token:
    head_token = sentence[token['head'] - 1]
    if head_token['form'] != '_' or head_token['id'] == 1:
        return head_token['id']
    else:
        for token in sentence:
            if token['head'] == head_token['id'] and token['deprel'] == '_':
                return token['id']

In [41]:
data = open("input_test.conllu", mode="r", encoding="utf-8")

sentences = parse(data.read())

#for i in range(len(sentences)):
sentence = sentences[0]

type(sentence)

conllu.models.TokenList